In [1]:
from auxiliary import loadData_staticTargetAddrMatch, prepare_data_loaders
from algorithms import RssPosAlgo_NeuralNet_MLP4layer
from algorithms import RssPosAlgo_NeuralNet_supervisedTrainingLoop
from algorithms import RssPosAlgo_NearestNeighbour_Interpolation
from algorithms import RssPosAlgo_NearestNeighbour_GetKmeansDb
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

### prepare model, dataloaders and training parameters

In [2]:
datajsonpath = "../experiments/exp002_20240907_sna_kadirburak/data-tshark/data_nosnap.json"
inp_rss_vals, gt_locations = loadData_staticTargetAddrMatch(datajsonpath, second_hold = 5, shuffle=False, 
                                                            target_addresses=["4c:77:6d:92:b3:60",
"4c:77:6d:5f:ea:e0",
"4c:77:6d:05:af:20"], snap250ms=False)

epochs           = 601
batch_size       = 8
train_test_split = 0.8

train_loader, test_loader, xtr, ytr, xts, yts = prepare_data_loaders(inp_rss_vals, gt_locations, 
                                                                     batch_size = 32, 
                                                                     train_test_split = train_test_split)

print("Subset sizes | train:", xtr.shape[0], ", test:",xts.shape[0])
MLP = RssPosAlgo_NeuralNet_MLP4layer()
MLP.train()

MLP_criterion = nn.MSELoss(reduction='mean')
MLP_optimizer = optim.Adam(MLP.parameters(), lr=3e-4)

Subset sizes | train: 289 , test: 73


### train model

In [3]:
MLP = RssPosAlgo_NeuralNet_supervisedTrainingLoop(train_loader = train_loader, 
                                                         test_loader  = test_loader,
                                                         model        = MLP, 
                                                         criterion    = MLP_criterion, 
                                                         optimizer    = MLP_optimizer, 
                                                         epochs       = epochs,
                                                         testfreq     = 20) # testfreq is in epochs

Epoch [1/601] test loss: 12.714, training loss: -1.000
Epoch [21/601] test loss: 0.341, training loss: 1.199
Epoch [41/601] test loss: 0.332, training loss: 1.164
Epoch [61/601] test loss: 0.321, training loss: 1.106
Epoch [81/601] test loss: 0.313, training loss: 1.063
Epoch [101/601] test loss: 0.309, training loss: 1.028
Epoch [121/601] test loss: 0.311, training loss: 1.001
Epoch [141/601] test loss: 0.314, training loss: 0.980
Epoch [161/601] test loss: 0.320, training loss: 0.965
Epoch [181/601] test loss: 0.325, training loss: 0.954
Epoch [201/601] test loss: 0.331, training loss: 0.946
Epoch [221/601] test loss: 0.336, training loss: 0.942
Epoch [241/601] test loss: 0.340, training loss: 0.936
Epoch [261/601] test loss: 0.342, training loss: 0.932
Epoch [281/601] test loss: 0.346, training loss: 0.934
Epoch [301/601] test loss: 0.347, training loss: 0.932
Epoch [321/601] test loss: 0.349, training loss: 0.930
Epoch [341/601] test loss: 0.350, training loss: 0.928
Epoch [361/601

### save the model

In [4]:
torch.save(MLP.state_dict(), 'savedmodels/dev006_MLP4layer.pth')

### evaluate model

In [5]:
MLP = RssPosAlgo_NeuralNet_MLP4layer()
MLP.load_state_dict(torch.load('savedmodels/dev006_MLP4layer.pth'));
MLP.eval();

In [6]:
db_kmeans = RssPosAlgo_NearestNeighbour_GetKmeansDb(xtr, ytr, num_clusters=5)
meanerror_nene_interp = 0;
meanerror_mlp         = 0;
for test_idx, x_test_sample in enumerate(xts): 
    loc_pred_mlp           = MLP(x_test_sample)
    loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(x_test_sample, db_kmeans)
    meanerror_mlp         += (yts[test_idx].numpy() - loc_pred_mlp.detach().numpy())**2
    meanerror_nene_interp += (yts[test_idx].numpy() - loc_pred_nene_interp)**2
print("MLP        :", np.linalg.norm(meanerror_mlp/(test_idx+1)))
print("NeNe+Interp:", np.linalg.norm(meanerror_nene_interp/(test_idx+1)))

MLP        : 0.6367153
NeNe+Interp: 0.9393942986245177


### test a few samples

from test set

In [7]:
sampleid = 13
loc_pred_mlp           = MLP(xts[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xts[sampleid], db_kmeans)
meanerror_mlp         += (yts[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (yts[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", yts[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [1.6728507 3.0964646]
MLP prediction   : [1.6975274 3.2904024]
NeNe+Interp pred : (1.6205665365850843, 3.03344483056108)


---

from train set

In [8]:
sampleid = 20
loc_pred_mlp           = MLP(xtr[sampleid])
loc_pred_nene_interp   = RssPosAlgo_NearestNeighbour_Interpolation(xtr[sampleid], db_kmeans)
meanerror_mlp         += (ytr[sampleid].numpy() - loc_pred_mlp.detach().numpy())**2
meanerror_nene_interp += (ytr[sampleid].numpy() - loc_pred_nene_interp)**2
print("Actual position  :", ytr[sampleid].numpy())
print("MLP prediction   :", loc_pred_mlp.detach().numpy())
print("NeNe+Interp pred :", loc_pred_nene_interp)

Actual position  : [2.4450567 3.1718736]
MLP prediction   : [1.7389529 2.5895522]
NeNe+Interp pred : (1.9227385956330416, 3.1851751698360373)
